In [2]:
from transformers import pipeline, AutoTokenizer
import pandas as pd
import json
from random import Random
from torch.utils.data import Dataset
from tqdm import tqdm
import transformers
import traceback
from Annotation import annotate
from copy import deepcopy

In [4]:
transformers.logging.set_verbosity_error()

ITEMS_PER_CONDITION = 10

class PromptDataset(Dataset):
    def __init__(self, prompts):
        self.prompts = prompts
    def __len__(self):
        return len(self.prompts)
    def __getitem__(self, idx):
        return self.prompts[idx]
        
models = [("stefan-it/german-gpt2-larger", 64, -1, None)]

with open("../items/names.json", encoding="utf-8") as nfile:
    namedict = json.load(nfile)
male_names = [name for name in namedict["male"]] * 2
female_names = [name for name in namedict["female"]] * 2
with open("../items/verbs_forced_reference.json", encoding="utf-8") as nfile:
    verbdict = json.load(nfile)
es_verbs = verbdict["es"]
se_verbs = verbdict["se"]

female_shuffled = female_names.copy()
Random(42).shuffle(female_shuffled)
male_shuffled = male_names.copy()
Random(84).shuffle(male_shuffled)
male_pairing = list(zip(male_names, female_shuffled, [False for name in male_names]))
female_pairing = list(zip(female_names, male_shuffled, [True for name in male_names]))

conditions = [
    (2,  es_verbs, female_pairing, "NP1"),
    (3,  es_verbs,   male_pairing, "NP1"),
    (6,  se_verbs, female_pairing, "NP1"),
    (7,  se_verbs,   male_pairing, "NP1"),
    (10, es_verbs, female_pairing, "NP2"),
    (11, es_verbs,   male_pairing, "NP2"),
    (14, se_verbs, female_pairing, "NP2"),
    (15, se_verbs,   male_pairing, "NP2")
]

items_per_condition = []
  
for condition, verbs, pairing, forced_reference in conditions:
    rows = []
    for verbdict in verbs:
        verb, filler, verbclass = verbdict["verb"], verbdict["filler"], verbdict["verbclass"]
        for np1, np2, female in pairing:
            prompt = f"{np1} {verb} {np2}{filler}, weil"
            nrow = {"condition": condition, "type": "Experiment", "prompt": prompt, "NP1": np1, "NP2": np2, 
                    "NP1gender": "f" if female else "m", "verb": verb, "verbclass": verbclass, "forced": forced_reference}
            rows.append(nrow)
    Random(168).shuffle(rows)
    items_per_condition.append(rows)
    
for model_name, batch_size, device, device_map in models:
       
    print(f"now loading: {model_name}")
    model = pipeline("text-generation", model = model_name, device = device, device_map = device_map)
    model.tokenizer.pad_token_id = model.model.config.eos_token_id
    model.tokenizer.padding_side = "left"


now loading: stefan-it/german-gpt2-larger


In [11]:
male_tokens = list(map(model.tokenizer.encode, [" er", " dieser", " jener", " der"]))
female_tokens = list(map(model.tokenizer.encode, [" sie", " diese", " jene", " die"]))

data = []

for condition in items_per_condition:
    items = deepcopy(condition)
    bar = tqdm(total = ITEMS_PER_CONDITION)
    item_iter = iter(items)
    rows = []
    while bar.n < ITEMS_PER_CONDITION:
        try:
            row = next(item_iter)
            if row["forced"] == "NP1":
                tokenized_name = model.tokenizer.encode(f" {row['NP1']}")
            else:
                tokenized_name = model.tokenizer.encode(f" {row['NP2']}")
            if (row["NP1gender"] == "m" and row["forced"] == "NP1") or (row["NP1gender"] == "f" and row["forced"] == "NP2"):
                print("Yo, waddup")
                forced_tokens = male_tokens + [tokenized_name]
            else:
                print("Cool story pro")
                forced_tokens = female_tokens + [tokenized_name]
            print(forced_tokens)
            continuation = model(row["prompt"], force_words_ids = [forced_tokens], remove_invalid_values=True, early_stopping = True, do_sample = False, num_beams = 2, max_new_tokens = 12)[0]["generated_text"]
            row["cont"] = continuation[len(row["prompt"]) + 1:]
            res = annotate(row, False)
            print(row["prompt"], row["cont"])
            print(row["forced"], row["NP1gender"], res["Koreferenz"])
            if res["Koreferenz"] == row["forced"]:
                bar.update(1)
                row.update(res)
                rows.append(row)
        except StopIteration:
            print(f"Run out of data in condition {items[0]['condition']}")
            break
    data += rows

exp3 = pd.DataFrame(data, columns = ["condition", "type", "prompt", "cont", "NP1", "NP2", "NP1gender", "verb", "verbclass", "forced", "Koreferenz", "Anaphorische Form"])
exp3.to_csv(f"../data/forced_coreference--{model_name.replace('/', '--')}.csv", sep=";", index=False)

del exp3
del rows
del items
    


  0%|                                                                                           | 0/10 [00:36<?, ?it/s]


Cool story pro
[[467], [846], [8924], [286], [7107, 2087]]



 10%|████████▎                                                                          | 1/10 [00:00<00:04,  2.15it/s]

Annika ekelte sich vor Kevin schon eine ganze Weile, weil Annika ihn nicht mochte.
Doch dann kam Kevin und
NP1 f NP1
Cool story pro
[[467], [846], [8924], [286], [34140]]
Melanie vergötterte Marcel vom ersten Moment an, weil er sich in sie verliebt hatte.
Doch dann kam alles
NP1 f NP2
Cool story pro
[[467], [846], [8924], [286], [16480]]
Katharina sorgte sich um Martin seit mehreren Tagen, weil er sich nicht mehr die Zähne putzen konnte.
Er
NP1 f NP2
Cool story pro
[[467], [846], [8924], [286], [17071]]
Sarah fürchtete Benjamin ein kleines bisschen, weil er sich nicht die Mühe machen wollte, sich mit dem Thema
NP1 f NP2
Cool story pro
[[467], [846], [8924], [286], [32501]]
Jessica bedauerte Benjamin aus tiefstem Herzen, weil er sich so sehr für die Kinder eingesetzt habe.
„
NP1 f NP2
Cool story pro
[[467], [846], [8924], [286], [2068, 8436]]
Luisa achtete Marco in hohem Grade, weil Luisa sich in den letzten Jahren immer mehr in den Vordergrund
NP1 f 
Cool story pro
[[467], [846], [8924


 20%|████████████████▌                                                                  | 2/10 [00:04<00:19,  2.49s/it]

...

Alina hasste Lars bis aufs Blut, weil Alina ihn nicht mochte.
Doch dann wurde Alina
NP1 f NP1


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:15<00:00,  1.56s/it]

Yo, waddup
[[333], [822], [10287], [282], [15730]]


Benjamin ekelte sich vor Alexandra schon eine ganze Weile, weil sie sich nicht mehr an der Leine halten konnte.
Sie
NP1 m NP2
Yo, waddup
[[333], [822], [10287], [282], [21268]]
Lars vergötterte Natalie vom ersten Moment an, weil sie sich in der Lage sah, sich in die Welt der
NP1 m NP2
Yo, waddup
[[333], [822], [10287], [282], [23253]]
Philip sorgte sich um Hannah seit mehreren Tagen, weil sie sich nicht mehr an der Schule aufhalten konnte.
Sie
NP1 m NP2
Yo, waddup
[[333], [822], [10287], [282], [6878]]
Daniel fürchtete Jacqueline ein kleines bisschen, weil sie sich nicht so gut erziehen lassen wollte.
Sie
NP1 m NP2
Yo, waddup
[[333], [822], [10287], [282], [15259]]


KeyboardInterrupt: 